In [4]:
from pulp import *
import pandas as pd
import numpy as np
from pandas import ExcelWriter

In [5]:
#Function para calcular o preço da passagem
def Tarifa(dist):
    return np.exp(-0.73953 * np.log(dist) + 4.44446)*dist

#Function para calcular o CASK
def Cask(dist, seats):
    return np.exp(1.728912 - 0.40453 * np.log(dist) - 0.35671 * np.log(seats))

In [6]:
# Leitura dos dados sobre aeronaves
  
DADOS3 = pd.read_excel('DadosTCC_TESTE.xlsx',sheet_name = 'Aeronaves') 
DADOS3.dropna(inplace=True)
DADOS3.reset_index(drop=True, inplace=True)
DADOS3.drop_duplicates(inplace=True)

AERONAVE = DADOS3['Aeronave']
N_ANV = len(AERONAVE)

ASSENTOS = DADOS3['Assentos']
    
ALCANCE = DADOS3['Alcance']

In [7]:
#Calculo do YIELD E CASK de HUB-HUB
#Método usando dictionary

DADOS5 = pd.read_excel(r'D:\GUSTAVO\UNB\TCC\UnB-tcc-gustavo-2021\Dados\Dados_TCC_2021.xlsx', sheet_name='Hubs')
Dist_hub ={}
tarifa_hub = {}
cask_hub ={}

for i in DADOS5['Hub Dist']:
#for i in HUBN:
    dic1 = {}
    dic2={}
    dic3 = {}
    #for j in range(len(DADOS5['Hub Dist'])):
    
    for j in range(len(DADOS5['Hub Dist'])):
        dic1[DADOS5['Hub Dist'][j]] = DADOS5[i][j]
        
        if i!= DADOS5['Hub Dist'][j]:
            dic2[DADOS5['Hub Dist'][j]] = Tarifa(DADOS5[i][j]) 
            
            dic4={}
            for anv in range(N_ANV):
                dic4[DADOS3['Aeronave'][anv]] = Cask(DADOS5[i][j], ASSENTOS[anv])
            dic3[DADOS5['Hub Dist'][j]] = dic4
            
    Dist_hub[i] = dic1
    tarifa_hub[i] = dic2
    cask_hub[i] = dic3

In [8]:
# EXEMPLO PARA O DATA_HUB - UTILIZANDO CENARIO HIPOTETICO

DF_exemplo = pd.DataFrame(data = {'HUB':[41,42,117,42,53,41,42,42], 
                     'Spoke': [53,53,53,117,117,117,117,117], 
                     'Demanda': [100,200,300,400,500,600,700,800],
                      'Distancia': [1000,2000,3000,4000,5000,6000,7000,8000]})

DF2_exemplo = pd.DataFrame(columns=['HUB', 'Spoke', 'Demanda','Distancia'])

list_hub = DF_exemplo['HUB'].drop_duplicates()
print('DF_exemplo:')
print(DF_exemplo)
print('list_hub:')
print(list_hub)

for hub in list_hub:
    print('HUB=', hub)
    DF2 = DF_exemplo[DF_exemplo['HUB']== hub]
    list_spoke = DF2['Spoke'].drop_duplicates().reset_index(drop=True)
    print('DF2:')
    print(DF2)
    print('list_spoke[HUB]:')
    print(list_spoke)
    
    for j in list_spoke:
        print('J=', j)
        DF3= DF2[DF2['Spoke']==j]
        soma = 0
        print('DF3')
        print(DF3)
        
        for i in DF3['Demanda']:
            soma += i
        DF4 = pd.DataFrame({'HUB':[hub], 'Spoke': [j], 'Demanda': [soma], 'Distancia': Dist_hub[hub][j]})
        print('DF4')
        print(DF4)
        DF2_exemplo = DF2_exemplo.append(DF4, ignore_index=True)
        print('DF2_exemplo')
        print(DF2_exemplo)
        
    

DF_exemplo:
   HUB  Spoke  Demanda  Distancia
0   41     53      100       1000
1   42     53      200       2000
2  117     53      300       3000
3   42    117      400       4000
4   53    117      500       5000
5   41    117      600       6000
6   42    117      700       7000
7   42    117      800       8000
list_hub:
0     41
1     42
2    117
4     53
Name: HUB, dtype: int64
HUB= 41
DF2:
   HUB  Spoke  Demanda  Distancia
0   41     53      100       1000
5   41    117      600       6000
list_spoke[HUB]:
0     53
1    117
Name: Spoke, dtype: int64
J= 53
DF3
   HUB  Spoke  Demanda  Distancia
0   41     53      100       1000
DF4
   HUB  Spoke  Demanda  Distancia
0   41     53      100       1068
DF2_exemplo
  HUB Spoke Demanda Distancia
0  41    53     100      1068
J= 117
DF3
   HUB  Spoke  Demanda  Distancia
5   41    117      600       6000
DF4
   HUB  Spoke  Demanda  Distancia
0   41    117      600       1118
DF2_exemplo
  HUB Spoke Demanda Distancia
0  41    53     100  